In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import List, Optional, Tuple

import numpy as np
from deepface import DeepFace


def cosine_distance(embedding1: List[float], embedding2: List[float]) -> float:
    """
    Calcula la distancia del coseno entre dos vectores de embedding.
    Devuelve un valor entre 0 (idénticos) y 2 (totalmente opuestos),
    aunque en la práctica se mueve mucho más cerca de 0 para caras similares.
    """
    v1 = np.array(embedding1, dtype=float)
    v2 = np.array(embedding2, dtype=float)

    denom = np.linalg.norm(v1) * np.linalg.norm(v2)
    if denom == 0:
        # Si algo sale mal con el vector, devolvemos la peor distancia posible
        return 1.0

    cos_sim = float(np.dot(v1, v2) / denom)
    # Distancia del coseno = 1 - similitud del coseno
    return 1.0 - cos_sim


@dataclass
class Person:
    """
    Representa una persona en la base:
    - id: identificador interno
    - name: nombre legible
    - embedding: vector numérico que representa el rostro
    """
    id: int
    name: str
    embedding: List[float]


class FaceDatabase:
    """
    Base de datos simple en memoria que almacena embeddings de personas
    y permite identificar una cara nueva comparando distancias.
    """

    def __init__(self, model_name: str = "Facenet"):
        """
        model_name: modelo interno que DeepFace usará para generar embeddings.
        Algunos valores posibles: 'VGG-Face', 'Facenet', 'ArcFace', 'OpenFace', etc.
        """
        self.people: List[Person] = []
        self.model_name = model_name

    def _embedding_from_image(self, image_path: str) -> List[float]:
        """
        Obtiene el embedding de una imagen usando DeepFace.represent.
        """
        reps = DeepFace.represent(
            img_path=image_path,
            model_name=self.model_name,
            enforce_detection=True,  # lanza error si no detecta cara
        )

        # DeepFace.represent devuelve una lista de diccionarios si hay varias caras;
        # aquí asumimos 1 sola cara en la imagen y usamos la primera.
        embedding = reps[0]["embedding"]
        return embedding

    def add_person_from_image(self, person_id: int, name: str, image_path: str) -> None:
        """
        Crea un embedding a partir de la imagen y lo guarda en la base.
        """
        embedding = self._embedding_from_image(image_path)
        person = Person(id=person_id, name=name, embedding=embedding)
        self.people.append(person)
        print(f"[INFO] Persona '{name}' (id={person_id}) añadida a la base.")

    def identify(self, image_path: str, threshold: float = 0.5) -> Tuple[Optional[Person], float]:
        """
        Intenta identificar a la persona en 'image_path'.

        - threshold: distancia máxima aceptada. Por debajo de esto consideramos que es la misma persona.
        - Devuelve:
            (persona_encontrada, distancia_minima)
          Si no coincide con nadie (distancia > threshold), persona_encontrada será None.
        """
        if not self.people:
            raise ValueError("La base de rostros está vacía. Agrega personas antes de identificar.")

        query_embedding = self._embedding_from_image(image_path)

        best_person: Optional[Person] = None
        best_distance: float = float("inf")

        for person in self.people:
            dist = cosine_distance(query_embedding, person.embedding)
            print(f"[DEBUG] Distancia a {person.name}: {dist:.4f}")
            if dist < best_distance:
                best_distance = dist
                best_person = person

        # Decisión según umbral
        if best_distance < threshold:
            return best_person, best_distance
        else:
            return None, best_distance
